In [ ]:
import matplotlib.dates as mdates
import numpy as np
import os
import pandas as pd
import rasterio as rs
import re
import seaborn as sns
import statsmodels.api as sm
import zipfile
from datetime import datetime

# get date, time and mean temperature per tile
directory = r"C:\Users\p_bl\Desktop\Studium_Master\15_Thesis\data\unzipped"
data_temp = []
for subfolder in os.listdir(directory):
    subfolder_directory = os.path.join(directory, subfolder)
    for file in os.listdir(subfolder_directory):
        if file.endswith("TABI.tiff"):
            # extract date and time
            match = re.search(r"\d{8}T\d{6}", file)
            datetime_string = match.group()
            time_object = datetime.strptime(datetime_string, '%Y%m%dT%H%M%S') # convert string to datetime-object
            # extract mean temperature
            with rs.open(os.path.join(subfolder_directory, file), "r") as tile:
                thermal = tile.read(1)
                thermal_masked = np.where(thermal == 0, np.nan, thermal) # mask nodata
                thermal_mean = np.nanmean(thermal_masked)
                thermal_median = np.nanmedian(thermal_masked)
            # append to list
            data_temp.append({
                "filename" : file,
                "datetime" : pd.Timestamp(time_object),
                "date" : pd.Timestamp(time_object).date(),
                "time" : pd.Timestamp(time_object).time(),
                "pxls" : (~np.isnan(thermal_masked)).sum(),
                "temp_mean": thermal_mean,
                "temp_median" : thermal_median,
            })
temp = pd.DataFrame(data_temp) 

# calculate weighted linear regression (wls)
temp['time_seconds'] = temp['time'].apply(lambda x: x.hour * 3600 + x.minute * 60 + x.second)
data_reg = []
for date in temp['date'].unique():
    # calculate wls
    subset = temp[temp['date'] == date]
    X = np.array(subset['time_seconds']).astype("int")
    X = sm.add_constant(X) # add constant to calculate intercept
    y = np.array(subset['temp_median']).astype("float")
    w = np.array(subset['pxls']).astype("int")
    wls_mod = sm.WLS(y, X, weights=w)
    wls_res = wls_mod.fit()
    # append wls parameters to list
    data_reg.append({
        "date": date,
        "intercept": wls_res.params[0],
        "slope": wls_res.params[1],
        "w_mean": (y * w).sum()/w.sum(),
        "pearson_r": subset["time_seconds"].corr(subset["temp_median"]),
        "r_squared": wls_res.rsquared,
        "p_value_slope": wls_res.pvalues[1],  
    })
reg = pd.DataFrame(data_reg) 
print(reg)

# plot results
for date in temp['date'].unique():
    subset = temp[temp['date'] == date]
    g = sns.relplot(
        data=subset, 
        x='datetime', 
        y='temp_median', 
        col='date',  
        marker="o",
        size="pxls", 
        color="#008B8B",
        col_wrap=None,
        facet_kws=dict(sharex=False, sharey=False), 
        legend='auto'
        );
    # calculate regression line
    intercept, slope = reg.loc[reg['date'] == date, ['intercept', 'slope']].iloc[0]
    x_pred = np.linspace(19 * 3600 + 50 * 60, 21 * 3600 + 30 * 60, 100) # start = 19:50, end = 21:30
    y_pred = intercept + slope * x_pred
    # adaptation of the plot output
    x_pred_datetime = pd.to_datetime(x_pred, unit='s', origin=pd.Timestamp(date))   
    ticks = pd.date_range(start=pd.Timestamp(f'{date} 20:00:00'), end=pd.Timestamp(f'{date} 21:30:00'), freq="15min")   
    for ax in g.axes.flat:
        ax.set_xticks(ticks)
        ax.xaxis.set_major_formatter(mdates.DateFormatter('%H:%M'))
        ax.set_ylabel("Brightness Temperature (K)", fontweight='bold')
        ax.set_xlabel("Time", fontweight='bold')
        ax.set_title("")
        ax.plot(x_pred_datetime, y_pred, color="red") 
    g.legend.set_title('Pixelanzahl')
    # save plot as png
    g.savefig(f"{date}.png", bbox_inches='tight')
    
# apply regression factor to data from 12.11.2022
read_dir = r"C:\Users\p_bl\Desktop\Studium_Master\15_Thesis\data\unzipped"
write_dir = r"C:\Users\p_bl\Desktop\Studium_Master\15_Thesis\data\time_adjusted"
for subfolder in os.listdir(read_dir):
    subfolder_directory = os.path.join(read_dir, subfolder)
    for file in os.listdir(subfolder_directory):
        if file.endswith("TABI.tiff") and file.startswith('20221112'):
            # calculate temperature delta
            temp_subset = temp[temp['filename'] == file]
            date_for_file = temp_subset['date'].iloc[0]
            reg_subset = reg[reg['date'] == date_for_file]
            min_time_seconds = temp[temp['date'] == date_for_file]['time_seconds'].min()
            time_delta = temp_subset['time_seconds'].iloc[0] - min_time_seconds
            temp_delta = time_delta * reg_subset['slope'].iloc[0]
            # subtract temperature delta and save files 
            with rs.open(os.path.join(subfolder_directory, file)) as tile:
                prod = tile.read()
                prod_adj = np.where(prod == 0, 0, prod - temp_delta)
                with rs.open(os.path.join(write_dir, file), 'w', **tile.meta) as tile_adj:
                    tile_adj.write(prod_adj)